In [78]:
import torch
import torch.nn as nn

In [79]:
dir(nn.Module)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_call_impl',
 '_get_backward_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_maybe_warn_non_full_backward_hook',
 '_named_members',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'get_buffer',
 'get_extra_state',
 'get_parameter',
 'get_submodule',
 'half',
 'ipu',

In [80]:
#importing libraries 
import numpy as np
import pandas as pd

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2

from tqdm import tqdm_notebook as tqdm
from functools import partial
import scipy as sp

import random
import time
import sys
import os
import yaml

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import confusion_matrix
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, models, datasets
from torchvision.models.feature_extraction import create_feature_extractor
from torch.utils.data import Dataset
from torch.autograd import Variable

import warnings
warnings.filterwarnings('ignore')

In [81]:
config_path = 'config/preprocessing_config.yaml'
conf = yaml.safe_load(open(config_path,'r'))
train_image_path = conf['train_dataset']['image_path']
test_image_path = conf['test_dataset']['image_path']
train_labels_path = conf['train_dataset']['label_path']
test_labels_path = conf['test_dataset']['label_path']
batch_size = conf['image_preprocessing']['batch_size']
shuffle = conf['image_preprocessing']['shuffle']
do_random_crop = conf['image_preprocessing']['do_random_crop']
columns = conf['image_preprocessing']['columns']
image_size = conf['image_preprocessing']['image_size']
itype = conf['image_preprocessing']['itype']
device = conf['metadata']['device']

In [82]:
from utils.dataloader import load_data
from utils.functions import *

In [83]:
# import data
train = pd.read_csv(train_labels_path)
test = pd.read_csv(test_labels_path)

# check shape
print(train.shape, test.shape)
print('-' * 15)
print(train[columns[1]].value_counts())
print('-' * 15)
print(test[columns[1]].value_counts())

(413, 12) (103, 3)
---------------
2    136
0    134
3     74
4     49
1     20
Name: Retinopathy grade, dtype: int64
---------------
0    34
2    32
3    19
4    13
1     5
Name: Retinopathy grade, dtype: int64


In [84]:
loader = load_data(train_labels_path, test_labels_path, train_image_path, test_image_path, columns, itype = '.jpg', batch_size = 16, shuffle=True, do_random_crop = False, device = 'cpu')
train_loader, test_loader, valid_loader = loader.create_loader()

In [85]:
# count = 0
# for batch in train_loader:
#     print(batch['image'].shape)
#     print(batch['label'].shape)
#     count = count+1
# print(count)
# count=0
# for batch in test_loader:
#     print(batch['image'].shape)
#     print(batch['label'].shape)
#     count=count+1
# print(count)
# count=0
# for batch in valid_loader:
#     print(batch['image'].shape)
#     print(batch['label'].shape)
#     count=count+1
# print(count)
# count=0

In [86]:
print(loader.valid_df.iloc[:5, 0:2])

   Image name  Retinopathy grade
51  IDRiD_052                  0
52  IDRiD_053                  4
53  IDRiD_054                  0
54  IDRiD_055                  0
55  IDRiD_056                  0


In [87]:
class RetinopathyClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
        
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
                      
            nn.AdaptiveAvgPool2d(output_size=(7, 7))
        )
    
        self.classifier = nn.Sequential(
            
            nn.Flatten(),  
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=1000, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=1000, out_features=5, bias=True),
            nn.Softmax()
        )
        
        self.overall = nn.Sequential(
                      
            self.features,
            self.avgpool,
            self.classifier
        )
        
    def forward(self, xb):
        
        self.out = self.overall(xb)
        return self.out

In [88]:
epochs = 10
lr = 1e-4
model = RetinopathyClassification()

In [ ]:
def fit(epochs, lr, model, train_loader, opt_func = torch.optim.Adam):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        result = {'train_loss': [], 'train_acc': []}
        train_losses = []
        train_accuracy = []
        for batch in train_loader:
            images, labels = batch['image'], batch['label']
            optimizer.zero_grad()
            out = model(images)                  # Generate predictions
            loss = ordinal_loss(out, labels) # Calculate loss
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            acc = ordinal_accuracy(out, labels)
            train_accuracy.append(acc)
            print("Epoch [{}], Batch [{}], train_loss: {:.4f}, train_acc: {:.4f}, val_acc: {:.4f}".format(
            epoch, batch, loss.detach(), acc.detach()), flush=True)
            result['train_loss'].append(loss.detach())
            result['train_acc'].append(acc.detach())
            
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['train_acc'] = torch.stack(train_accuracy).mean().item()
        print("Epoch [{}], train_loss: {:.4f}, train_acc: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['train_acc']))
        history.append(result)
    
    return history

In [ ]:
fit(epochs, lr, model, train_loader)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/sanskriti/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_6582/4265229552.py", line 1, in <module>
    fit(epochs, lr, model, train_loader)
  File "/tmp/ipykernel_6582/1978807601.py", line 15, in fit
    loss = ordinal_loss(out, labels) # Calculate loss
  File "/home/sanskriti/qtm_comp/utils/functions.py", line 19, in ordinal_loss
  File "/home/sanskriti/qtm_comp/utils/functions.py", line 11, in loss_util
    loss += torch.log(torch.exp(y_pred[:, i]) + 1e-10).sum() - y_pred[:, i + 1].sum()
IndexError: too many indices for tensor of dimension 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sanskriti/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceb